In [32]:
import geopandas as gpd

houses = gpd.read_file("/Users/test/Documents/code/gcn_prov/Обеспеченность сервисами _ Детский сад,Жилые дома.geojson").rename(columns={'id':'building_id'}).reset_index(drop=True).reset_index(drop=False).rename(columns={'index':'bid', 'kindergartens_service_demand_value_normative': 'demand_population', "kindergartens_supplyed_demands_within": "demand_in"})
services = gpd.read_file("/Users/test/Documents/code/gcn_prov/Обеспеченность сервисами _ Детский сад,Сервисы.geojson").rename(columns={'id':'service_id'}).reset_index(drop=True).reset_index(drop=False).rename(columns={'index':'sid'})
connections = gpd.read_file("/Users/test/Documents/code/gcn_prov/Обеспеченность сервисами _ Детский сад,Детский сад, связи.geojson").dropna(subset='geometry')




"HERE"
connections = connections[connections['demand']>1].reset_index(drop=True).reset_index(drop=False).rename(columns={'index':'cid'})
"HERE"

# services['capacity'] /= 3


# Convert CRS from EPSG 4326 to EPSG 32636
houses = houses.to_crs(epsg=32636)
services = services.to_crs(epsg=32636)
connections = connections.to_crs(epsg=32636)

connections['distance'] = connections['geometry'].length.round(0).astype(int)

connections = connections.merge(houses[['bid', "building_id", 'demand_in', 'demand_population']]).merge(services[['sid', 'service_id']])

service_features = services[['sid', 'capacity']].drop_duplicates().reset_index(drop=True)

# connections = connections[connections['demand_in']>0]
connections = connections[connections['distance']<5000]


In [33]:
connections.explore(column='demand')

In [34]:
connections2 = connections.merge(service_features, on='sid').copy()
connections2.loc[connections2['distance'] < 10, 'distance'] = 10
connections2['grav'] = connections2['demand_population'] * connections2["capacity"]/ connections2['distance']**2
# connections2['not_supplyed'] = connections2['demand_population'] - connections2['demand_in']

In [35]:
connections2

,cid,id,building_id,demand,service_id,geometry,distance,bid,demand_in,demand_population,sid,capacity,grav
0,10,1268,1721011,3,145821,"LINESTRING (346506.810 6649852.507, 349467.182...",3068,1310,0,35,107,590,0.002194
1,13,2798,45717,3,145842,"LINESTRING (347439.041 6647774.778, 347662.819...",225,717,0,11,62,160,0.034765
2,14,2799,45834,3,145842,"LINESTRING (347460.045 6647903.330, 347662.819...",227,728,3,4,62,160,0.012420
3,15,2800,45944,2,145842,"LINESTRING (347473.355 6647809.197, 347662.819...",190,745,0,3,62,160,0.013296
4,16,2801,46005,2,145842,"LINESTRING (347478.065 6647887.427, 347662.819...",203,750,2,2,62,160,0.007765
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,1884,91169,1721128,2,147083,"LINESTRING (343041.866 6647455.217, 345597.936...",2562,1336,0,24,165,146,0.000534
1730,1885,91175,1721148,7,147083,"LINESTRING (345203.341 6647429.672, 345597.936...",420,1356,0,49,165,146,0.040556
1731,1886,91176,1721152,2,147083,"LINESTRING (347239.336 6646534.968, 345597.936...",1805,1359,0,28,165,146,0.001255
1732,1887,91177,1721153,3,147083,"LINESTRING (347194.171 6646485.986, 345597.936...",1785,1360,0,47,165,146,0.002154


In [36]:
connections2.drop(columns=['geometry']).groupby('service_id')['capacity'].mean().sum()

18111.0

In [37]:
connections2['demand'].sum()

8720

In [7]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# # Define dataset
# class FlowDataset(Dataset):
#     def __init__(self, demand, service, distances, flows):
#         self.demand = demand
#         self.service = service
#         self.distances = distances
#         self.flows = flows
#         # self.grav = grav

#     def __len__(self):
#         return len(self.flows)

#     def __getitem__(self, idx):
#         return self.demand[idx], self.service[idx], self.distances[idx], self.flows[idx]

# # Define the Physics Aware Neural Network
# class PhysicsAwareNN(nn.Module):
#     def __init__(self):
#         super(PhysicsAwareNN, self).__init__()
#         self.fc1 = nn.Linear(3, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 1)

#     def forward(self, demand, service, distance):
#         x = torch.cat((demand, service, distance), dim=1)
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# # Custom loss function to incorporate gravity law
# def gravity_loss(predicted_flows, actual_flows,
#                  conn_df):
#     conn_df = conn_df.copy()
#     # gravity_term = (predicted_flows * service) / (distance ** 2)
#     # loss1 = torch.nn.MSELoss()(gravity_term, grav)
#     loss2 = torch.nn.MSELoss()(predicted_flows, actual_flows)

#     conn_df['predicted'] = predicted_flows.detach().numpy()
#     penalty = conn_df.groupby('service_id')['capacity'].mean() - conn_df.groupby('service_id')['predicted'].sum()
#     penalty = torch.tensor(penalty.values, dtype=torch.float32, device=predicted_flows.device).median()

#     penalty2 = conn_df.groupby('building_id')['demand_population'].mean() - conn_df.groupby('building_id')['predicted'].sum()
#     penalty2 = torch.tensor(penalty2.values, dtype=torch.float32, device=predicted_flows.device).median()


#     return loss2*5 + penalty/500 + penalty2

# # Load data
# demand_data = torch.tensor(connections2[['demand_population']].values, dtype=torch.float32)  # Load demand data
# service_data = torch.tensor(connections2[['capacity']].values, dtype=torch.float32)  # Load service data
# distances = torch.tensor(connections2[['distance']].values, dtype=torch.float32)  # Load distances data
# # grav = torch.tensor(connections2[['grav']].values, dtype=torch.float32)  # Load distances data

# flows = torch.tensor(connections2[['demand']].values, dtype=torch.float32)  # Load flows data

# # Create dataset and dataloader
# dataset = FlowDataset(demand_data, service_data, distances, flows)
# # dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Initialize model, optimizer, and loss function
# model = PhysicsAwareNN()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# from torch_geometric.data import Data
# import torch

# def make_data(connections):

#     connections = connections2.copy()
#     # Создание узлов и ребер
#     node_features = []  # Здесь будут храниться признаки узлов
#     edges = []  # Список ребер
#     edge_attrs = []  # Список признаков ребер

#     # Предположим, что каждый узел - это building_id, а ребра между ними - это связи
#     building_ids = connections['building_id'].unique()
#     service_ids = connections['service_id'].unique()

#     building_id_map = {bid: idx for idx, bid in enumerate(building_ids)}
#     service_id_map = {sid: idx for idx, sid in enumerate(service_ids)}


#     # Заполнение узлов признаками
#     for building_id in building_ids:
#         demand = connections[connections['building_id'] == building_id]['demand'].mean()
#         capacity = connections[connections['building_id'] == building_id]['capacity'].mean()
#         node_features.append([demand, capacity])

#     node_features = torch.tensor(node_features, dtype=torch.float)

#     # Заполнение ребер и их признаков
#     for index, row in connections.iterrows():
#         src = building_id_map[row['building_id']]
#         dst = service_id_map[row['service_id']]  # Предполагается, что 'bid' - это другой конец связи
#         distance = row['distance']
#         service_id = row['service_id']
        
#         edges.append([src, dst])
#         edge_attrs.append([distance, service_id])

#     edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
#     edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

#     y = torch.tensor(connections['demand'], dtype=torch.float)

#     # Создание объекта Data для PyTorch Geometric
#     data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y)

#     return data

# data = make_data(connections)
# print(data)


In [38]:
connections

,cid,id,building_id,demand,service_id,geometry,distance,bid,demand_in,demand_population,sid
10,10,1268,1721011,3,145821,"LINESTRING (346506.810 6649852.507, 349467.182...",3068,1310,0,35,107
13,13,2798,45717,3,145842,"LINESTRING (347439.041 6647774.778, 347662.819...",225,717,0,11,62
14,14,2799,45834,3,145842,"LINESTRING (347460.045 6647903.330, 347662.819...",227,728,3,4,62
15,15,2800,45944,2,145842,"LINESTRING (347473.355 6647809.197, 347662.819...",190,745,0,3,62
16,16,2801,46005,2,145842,"LINESTRING (347478.065 6647887.427, 347662.819...",203,750,2,2,62
...,...,...,...,...,...,...,...,...,...,...,...
1884,1884,91169,1721128,2,147083,"LINESTRING (343041.866 6647455.217, 345597.936...",2562,1336,0,24,165
1885,1885,91175,1721148,7,147083,"LINESTRING (345203.341 6647429.672, 345597.936...",420,1356,0,49,165
1886,1886,91176,1721152,2,147083,"LINESTRING (347239.336 6646534.968, 345597.936...",1805,1359,0,28,165
1887,1887,91177,1721153,3,147083,"LINESTRING (347194.171 6646485.986, 345597.936...",1785,1360,0,47,165


In [39]:
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
import numpy as np

def make_data(connections):
    connections = connections.copy()
    connections['grav'] = connections['demand_population'] * connections["capacity"]/ connections['distance']**2

    # Unique identifiers for buildings and services
    building_ids = connections['building_id'].unique()
    service_ids = connections['service_id'].unique()

    # Maps to convert building and service IDs to indices
    building_id_map = {bid: idx for idx, bid in enumerate(building_ids)}
    service_id_map = {sid: idx for idx, sid in enumerate(service_ids)}

    # Create node features (demand and capacity for each building)
    node_features = []
    for building_id in building_ids:
        demand = connections[connections['building_id'] == building_id]['demand_population'].mean()
        capacity = connections[connections['building_id'] == building_id]['capacity'].mean()
        distance = connections[connections['building_id'] == building_id]['distance'].mean()
        node_features.append([demand, capacity, distance])

    # Convert node features to a numpy array for scaling
    node_features = np.array(node_features)

    # Initialize the scaler
    scaler = StandardScaler()

    # Fit and transform the node features
    node_features_scaled = scaler.fit_transform(node_features)

    # Convert the scaled features back to a torch tensor
    node_features = torch.tensor(node_features_scaled, dtype=torch.float)
    # Create edges and edge attributes (distance and service ID for each connection)
    edges = []
    edge_attrs = []
    for index, row in connections.iterrows():
        src = building_id_map[row['building_id']]
        dst = service_id_map[row['service_id']]
        distance = row['grav']
        # service_id = row['service_id']
        
        edges.append([src, dst])
        edge_attrs.append([distance])

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

    # Create target tensor (demand for each connection)
    y = torch.tensor(connections['demand'].values, dtype=torch.float)

    # Create Data object for PyTorch Geometric
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y)

    return data

data = make_data(connections2)

In [68]:
import numpy as np
import torch
import pandas as pd
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class FlowDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

# Define GNN model
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(3, 128)
        self.conv2 = GCNConv(128, 64)
        self.fc = torch.nn.Linear(128, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        
        # x = torch.mean(x, dim=0)
               # Concatenate the node features of the two nodes for each edge
        row, col = edge_index
        edge_features = torch.cat([x[row], x[col]], dim=1)
        
        # Predict the edge weights
        edge_weights = self.fc(edge_features).squeeze()
        return edge_weights
        # return x

# Custom loss function to incorporate gravity law and capacity constraints
def gravity_loss(predicted_flows, actual_flows, conn_df, penalty_weight=1.0):
    conn_df = conn_df.copy()
    conn_df['predicted'] = predicted_flows.detach().cpu().numpy()
    
    penalty_service = conn_df.groupby('service_id')['predicted'].sum() - conn_df.groupby('service_id')['capacity'].mean()
    penalty_service = torch.tensor(penalty_service.values, dtype=torch.float32, device=predicted_flows.device)
    penalty_service = torch.where(penalty_service > 0, penalty_service, torch.zeros_like(penalty_service)).mean()  # Penalty only if sum of predictions exceeds capacity

    penalty_building = conn_df.groupby('building_id')['demand_population'].mean() - conn_df.groupby('building_id')['predicted'].sum()
    penalty_building = torch.tensor(penalty_building.values, dtype=torch.float32, device=predicted_flows.device)
    penalty_building = torch.where(penalty_building > 0, torch.zeros_like(penalty_building), -penalty_building).mean()  # Penalty only if sum of predictions is less than demand_population

    loss2 = torch.nn.MSELoss()(predicted_flows, actual_flows)
    penalty_service *= penalty_weight
    penalty_building *= penalty_weight
    
    return loss2 + penalty_service + penalty_building

import torch.optim as optim
from torch_geometric.data import DataLoader

# Create your dataset object
# Assuming your 'data' object already exists and contains all necessary data
dataset = [data]  # If you have multiple graphs, add them to this list
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

model = GNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(epoch):
    model.train()
    for data in dataloader:
        optimizer.zero_grad()
        predicted_flows = model(data)

        loss = gravity_loss(predicted_flows=predicted_flows, actual_flows=data.y, conn_df=connections2)
        loss.backward()
        optimizer.step()

        # Calculate metrics
        actual_flows_np = data.y.detach().cpu().numpy()
        predicted_flows_np = predicted_flows.detach().cpu().numpy()
        
        mae = round(mean_absolute_error(y_true=actual_flows_np, y_pred=predicted_flows_np),3)
        r2 = round(r2_score(actual_flows_np, predicted_flows_np),3)

        print(f"Epoch {epoch+1}, Loss: {round(loss.item(),3)}, MAE: {round(mae,3)}, R2: {r2}")

for epoch in range(1500):  # You can change the number of epochs
    train(epoch)


/Users/test/Documents/code/.venv/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 72.202, MAE: 4.84499979019165, R2: -0.477
Epoch 2, Loss: 70.643, MAE: 4.679999828338623, R2: -0.445
Epoch 3, Loss: 69.137, MAE: 4.513999938964844, R2: -0.414
Epoch 4, Loss: 67.667, MAE: 4.34499979019165, R2: -0.384
Epoch 5, Loss: 66.244, MAE: 4.182000160217285, R2: -0.355
Epoch 6, Loss: 64.858, MAE: 4.0229997634887695, R2: -0.326
Epoch 7, Loss: 63.51, MAE: 3.86299991607666, R2: -0.299
Epoch 8, Loss: 62.218, MAE: 3.7119998931884766, R2: -0.272
Epoch 9, Loss: 60.985, MAE: 3.5829999446868896, R2: -0.247
Epoch 10, Loss: 59.808, MAE: 3.4670000076293945, R2: -0.222
Epoch 11, Loss: 58.694, MAE: 3.374000072479248, R2: -0.199
Epoch 12, Loss: 57.654, MAE: 3.305999994277954, R2: -0.177
Epoch 13, Loss: 56.701, MAE: 3.265000104904175, R2: -0.157
Epoch 14, Loss: 55.841, MAE: 3.246000051498413, R2: -0.138
Epoch 15, Loss: 55.089, MAE: 3.246000051498413, R2: -0.122
Epoch 16, Loss: 54.71, MAE: 3.259999990463257, R2: -0.107
Epoch 17, Loss: 55.002, MAE: 3.2920000553131104, R2: -0.095
Epoch 

In [12]:


# # Training loop
# for epoch in range(1000):
#     # for demand, service, distance, actual_flows in dataloader:
#     optimizer.zero_grad()
#     predicted_flows = model(dataset.demand, dataset.service, dataset.distances)
#     loss = gravity_loss(predicted_flows, dataset.flows, connections2)
#     loss.backward()
#     optimizer.step()

#     mse = mean_squared_error(dataset.flows.detach().numpy(), predicted_flows.detach().numpy())
#     mae = mean_absolute_error(dataset.flows.detach().numpy(), predicted_flows.detach().numpy())
#     r2 = r2_score(dataset.flows.detach().numpy(), predicted_flows.detach().numpy())

#     print(f"Epoch {epoch+1}, Loss: {loss.item()}, Validation MSE: {mse}, MAE: {mae}, R2: {r2}")
# # Save the trained model
# torch.save(model.state_dict(), "physics_aware_nn.pth")


In [87]:
model.eval()
with torch.no_grad():
    predicted_flows = model(data)

connections3 = connections2.copy()
connections3['pred'] = predicted_flows.detach().numpy()
connections3['diff1'] = connections3['demand_in'] - connections3['pred']

# Load data
# connections3.loc[connections3['building_id']==36204,'demand_population']+=100
connections3.loc[connections3['service_id']==146851,'capacity'] +=200
connections3.loc[connections3['service_id']==147069,'capacity'] +=200



data2 = make_data(connections3)

model.eval()
with torch.no_grad():
    predicted_flows2 = model(data2)

connections3['pred2'] = predicted_flows2.detach().numpy()
connections3['diff2'] = connections3['demand_in'] - connections3['pred2']
connections3['diff21'] = connections3['pred2'] - connections3['pred']

# connections3 = connections3.merge(, on='service_id', how='left')


connections3

,cid,id,building_id,demand,service_id,geometry,distance,bid,demand_in,demand_population,sid,capacity,grav,pred,diff1,pred2,diff2,diff21
0,10,1268,1721011,3,145821,"LINESTRING (346506.810 6649852.507, 349467.182...",3068,1310,0,35,107,590,0.002194,4.200211,-4.200211,4.136595,-4.136595,-0.063616
1,13,2798,45717,3,145842,"LINESTRING (347439.041 6647774.778, 347662.819...",225,717,0,11,62,160,0.034765,0.881527,-0.881527,0.826150,-0.826150,-0.055377
2,14,2799,45834,3,145842,"LINESTRING (347460.045 6647903.330, 347662.819...",227,728,3,4,62,160,0.012420,-0.287632,3.287632,-0.932737,3.932737,-0.645105
3,15,2800,45944,2,145842,"LINESTRING (347473.355 6647809.197, 347662.819...",190,745,0,3,62,160,0.013296,0.494737,-0.494737,0.383148,-0.383148,-0.111589
4,16,2801,46005,2,145842,"LINESTRING (347478.065 6647887.427, 347662.819...",203,750,2,2,62,160,0.007765,2.663957,-0.663957,2.643432,-0.643432,-0.020525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,1884,91169,1721128,2,147083,"LINESTRING (343041.866 6647455.217, 345597.936...",2562,1336,0,24,165,146,0.000534,1.531977,-1.531977,1.425810,-1.425810,-0.106167
1730,1885,91175,1721148,7,147083,"LINESTRING (345203.341 6647429.672, 345597.936...",420,1356,0,49,165,146,0.040556,12.542252,-12.542252,12.443457,-12.443457,-0.098795
1731,1886,91176,1721152,2,147083,"LINESTRING (347239.336 6646534.968, 345597.936...",1805,1359,0,28,165,146,0.001255,1.663191,-1.663191,1.559118,-1.559118,-0.104074
1732,1887,91177,1721153,3,147083,"LINESTRING (347194.171 6646485.986, 345597.936...",1785,1360,0,47,165,146,0.002154,1.994751,-1.994751,1.875520,-1.875520,-0.119231


In [92]:
services.merge((connections2.groupby('service_id')['capacity'].mean() - connections3.groupby('service_id')['pred'].sum()).reset_index(), on='service_id').rename(columns={0:'FREE CAPACITY BEFORE'})[['geometry', 'FREE CAPACITY BEFORE']].explore(column='FREE CAPACITY BEFORE',     style_kwds={"radius":10}, tiles="CartoDB positron", scheme='Quantiles'
)

In [93]:
services.merge((connections3.groupby('service_id')['capacity'].mean() - connections3.groupby('service_id')['pred2'].sum()).reset_index(), on='service_id').rename(columns={0:'FREE CAPACITY AFTER INCREASE'})[['geometry', 'FREE CAPACITY AFTER INCREASE']].explore(column='FREE CAPACITY AFTER INCREASE',     style_kwds={"radius":10}, tiles="CartoDB positron", scheme='Quantiles'
)

In [90]:
col = 'diff21'

connections3.loc[connections3['diff21'].abs()>2, ['capacity', 'service_id', 'building_id', 'demand_population', 'demand', 'geometry', "pred","pred2", 'diff21']].explore(column=col, tiles="CartoDB positron", highlight=False, legend=True,  ontrol_scale=True, scheme='Quantiles')